In [ ]:
import pandas as pd
import numpy as np
import time

# 创建一个比之前大一些的DataFrame，模拟真实数据
rows = 5_000_000  # 500万行

# 步骤 1: 先创建基础的、不相互依赖的列
df = pd.DataFrame(
    {
        "open_time": pd.to_datetime(np.arange(rows), unit="s", origin="2020-01-01"),
        "open": 100 + np.random.randn(rows).cumsum(),
        # lambda 函数会被当作值来处理，而不是作为计算函数，所以在这里不能使用
        # "high": lambda x: x["open"] + np.random.rand(rows) * 5,
        # "low": lambda x: x["open"] - np.random.rand(rows) * 5,
        # "close": lambda x: x["open"] + np.random.randn(rows),
        "volume": np.random.randint(100, 5000, size=rows),
    }
)
# 步骤 2: 基于已经创建的 'open' 列，来创建其他列
# 这样可以保证我们是在用数值进行计算
df["high"] = df["open"] + np.random.rand(rows) * 5
df["low"] = df["open"] - np.random.rand(rows) * 5
df["close"] = df["open"] + np.random.randn(rows)

# 步骤 3: 最后设置索引
df.set_index("open_time", inplace=True)

print(f"创建了一个 {len(df)} 行的DataFrame。")

# --- 写入测试 ---
# 1. 写入 CSV
start_time = time.time()
df.to_csv("2025-08-22-001-large_data.csv")
print(f"写入CSV耗时: {time.time() - start_time:.4f} 秒")

# 2. 写入 Feather
start_time = time.time()
df.reset_index().to_feather("2025-08-22-001-large_data.feather")  # Feather需要重置索引
print(f"写入Feather耗时: {time.time() - start_time:.4f} 秒")

# 3. 写入 Parquet
start_time = time.time()
df.to_parquet("2025-08-22-001-large_data.parquet")
print(f"写入Parquet耗时: {time.time() - start_time:.4f} 秒")


print("\n--- 读取测试 ---")
# --- 读取测试 ---
# 1. 读取 CSV
start_time = time.time()
df_csv = pd.read_csv(
    "2025-08-22-001-large_data.csv", parse_dates=["open_time"]
).set_index("open_time")
print(f"读取CSV耗时: {time.time() - start_time:.4f} 秒")

# 2. 读取 Feather
start_time = time.time()
df_feather = pd.read_feather("2025-08-22-001-large_data.feather").set_index("open_time")
print(f"读取Feather耗时: {time.time() - start_time:.4f} 秒")

# 3. 读取 Parquet
start_time = time.time()
df_parquet = pd.read_parquet("2025-08-22-001-large_data.parquet")
print(f"读取Parquet耗时: {time.time() - start_time:.4f} 秒")


print("\n--- 列读取测试 (只读取 'close' 和 'volume') ---")
# --- 只读取部分列的测试 ---
# 1. 读取 CSV (仍然需要读取大部分内容)
start_time = time.time()
df_csv_cols = pd.read_csv(
    "2025-08-22-001-large_data.csv",
    usecols=["open_time", "close", "volume"],
    parse_dates=["open_time"],
).set_index("open_time")
print(f"从CSV读取部分列耗时: {time.time() - start_time:.4f} 秒")

# 2. 读取 Parquet (优势体现)
start_time = time.time()
df_parquet_cols = pd.read_parquet(
    "2025-08-22-001-large_data.parquet", columns=["close", "volume"]
)
print(f"从Parquet读取部分列耗时: {time.time() - start_time:.4f} 秒")

start_time = time.time()
df_feather_cols = pd.read_feather(
    "2025-08-22-001-large_data.feather", columns=["close", "volume"]
)
print(f"从Feather读取部分列耗时: {time.time() - start_time:.4f} 秒")


--- 读取测试 ---
读取CSV耗时: 9.5929 秒
读取Feather耗时: 0.3301 秒
读取Parquet耗时: 0.2691 秒

--- 列读取测试 (只读取 'close' 和 'volume') ---
从CSV读取部分列耗时: 8.6969 秒
从Parquet读取部分列耗时: 0.1851 秒
从Feather读取部分列耗时: 0.1483 秒


创建了一个 5000000 行的DataFrame。

写入CSV耗时: 37.4482 秒

写入Feather耗时: 0.6022 秒

写入Parquet耗时: 1.3751 秒


--- 读取测试 ---

读取CSV耗时: 10.0400 秒

读取Feather耗时: 0.3473 秒

读取Parquet耗时: 0.3186 秒

--- 列读取测试 (只读取 'close' 和 'volume') ---

从CSV读取部分列耗时: 8.9447 秒

从Parquet读取部分列耗时: 0.1691 秒